In [261]:
import pandas as pd
import os
import glob
from ddf_utils.str import to_concept_id, format_float_digits

In [262]:
source_dir = "../source/"
os.chdir(source_dir)

In [263]:

#source_files_temp = os.popen('ls -m *.csv', 'r').read().split(',')

In [264]:
source_files_temp = glob.glob('[0-9]*.csv')

In [265]:
source_files_temp.sort()

In [266]:
source_files_map = map(lambda x:x.strip(),source_files_temp)

In [267]:
source_files = list(source_files_map)

In [268]:
source_files

['0000.csv',
 '0001.csv',
 '0002.csv',
 '0003.csv',
 '0004.csv',
 '0005.csv',
 '0006.csv',
 '0007.csv',
 '0008.csv',
 '0009.csv',
 '0010.csv',
 '0011.csv',
 '0012.csv',
 '0013.csv',
 '0014.csv',
 '0015.csv',
 '0016.csv',
 '0017.csv',
 '0018.csv',
 '0019.csv',
 '0020.csv',
 '0021.csv',
 '0022.csv',
 '0023.csv',
 '0024.csv',
 '0025.csv',
 '0026.csv',
 '0027.csv',
 '0028.csv',
 '0029.csv',
 '0030.csv',
 '0031.csv',
 '0032.csv',
 '0033.csv',
 '0034.csv',
 '0035.csv',
 '0036.csv',
 '0037.csv',
 '0038.csv',
 '0039.csv',
 '0040.csv',
 '0041.csv',
 '0042.csv',
 '0043.csv',
 '0044.csv',
 '0045.csv',
 '0046.csv',
 '0047.csv',
 '0048.csv',
 '0049.csv',
 '0050.csv',
 '0051.csv',
 '0052.csv',
 '0053.csv',
 '0054.csv',
 '0055.csv',
 '0056.csv']

In [269]:
df_master = pd.DataFrame()

In [270]:
df_bracket_first = pd.read_csv("../source/" + source_files[0],header=0, usecols=['CountryCode','CountryName','CoverageType','RequestYear',\
                                                      'PovertyLine', 'HeadCount','ReqYearPopulation'])

In [271]:
df_master = df_master.append(df_bracket_first) #First most Bracket 0.00781 is simply appended to the data without any subtraction
del df_bracket_first


In [272]:
for i in range(1,len(source_files)):    
    df_bracket_start = pd.read_csv("../source/" + source_files[i-1],header=0, usecols=['CountryCode','CountryName','CoverageType','RequestYear',\
                                                      'PovertyLine', 'HeadCount','ReqYearPopulation'])
    df_bracket_end = pd.read_csv("../source/" + source_files[i],header=0, usecols=['CountryCode','CountryName','CoverageType','RequestYear',\
                                                      'PovertyLine', 'HeadCount','ReqYearPopulation'])
    df_bracket_end['HeadCount'] = df_bracket_end['HeadCount'] - df_bracket_start['HeadCount']
    
    df_master = df_master.append(df_bracket_end)
    del df_bracket_start
    del df_bracket_end
    

In [273]:
#df_bracket_end['Population'] = df_bracket_end['HeadCount'] * df_bracket_end['ReqYearPopulation'] * 1000000

In [274]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378765 entries, 0 to 6644
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CountryCode        378765 non-null  object 
 1   CountryName        378765 non-null  object 
 2   CoverageType       378765 non-null  object 
 3   RequestYear        378765 non-null  int64  
 4   PovertyLine        378765 non-null  float64
 5   HeadCount          378423 non-null  float64
 6   ReqYearPopulation  378765 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 23.1+ MB


In [275]:
df_master.head(100)

,CountryCode,CountryName,CoverageType,RequestYear,PovertyLine,HeadCount,ReqYearPopulation
0,ALB,Albania,N,1981,0.00781,0.000000,2.726056
1,DZA,Algeria,N,1981,0.00781,0.000000,19.824301
2,AGO,Angola,N,1981,0.00781,0.000000,8.640446
3,ARG,Argentina,U,1981,0.00781,0.002289,23.609667
4,ARM,Armenia,N,1981,0.00781,0.000130,3.148096
...,...,...,...,...,...,...,...
95,MLI,Mali,N,1981,0.00781,0.000000,7.234728
96,MLT,Malta,N,1981,0.00781,0.000000,0.318982
97,MRT,Mauritania,N,1981,0.00781,0.000000,1.586493
98,MUS,Mauritius,N,1981,0.00781,0.000000,0.980462


In [276]:
df_master['Population'] = df_master['HeadCount'] * df_master['ReqYearPopulation'] * 1000000

In [277]:
df_master.drop(columns=['CountryCode', 'HeadCount', 'ReqYearPopulation'], inplace=True)

In [278]:
df_master.head(100)

,CountryName,CoverageType,RequestYear,PovertyLine,Population
0,Albania,N,1981,0.00781,0.000000
1,Algeria,N,1981,0.00781,0.000000
2,Angola,N,1981,0.00781,0.000000
3,Argentina,U,1981,0.00781,54040.638990
4,Armenia,N,1981,0.00781,410.742789
...,...,...,...,...,...
95,Mali,N,1981,0.00781,0.000000
96,Malta,N,1981,0.00781,0.000000
97,Mauritania,N,1981,0.00781,0.000000
98,Mauritius,N,1981,0.00781,0.000000


In [279]:
def preprocess(data):
    """preprocessing the data:
    1. rename the first column to geo_name and third column to year
    2. rename the geo_name to alphanumeric
    3. drop all empty lines

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = data.rename(columns={data.columns[0]: 'geo_name', data.columns[2]: 'Year', data.columns[3]: 'income_bracket'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    data = data.reset_index()
    return data

In [280]:
df_master = preprocess(df_master)

In [281]:
df_master.head(10)

,geo,geo_name,CoverageType,Year,income_bracket,Population
0,albania,Albania,N,1981,0.00781,0.000000
1,algeria,Algeria,N,1981,0.00781,0.000000
2,angola,Angola,N,1981,0.00781,0.000000
3,argentina,Argentina,U,1981,0.00781,54040.638990
4,armenia,Armenia,N,1981,0.00781,410.742789
5,australia,Australia,N,1981,0.00781,0.000000
6,austria,Austria,N,1981,0.00781,0.000000
7,azerbaijan,Azerbaijan,N,1981,0.00781,0.000000
8,bangladesh,Bangladesh,N,1981,0.00781,30409.625015
9,belarus,Belarus,N,1981,0.00781,0.000000
